# Basic Search

The CDA provides a custom python tool for searching CDA data. [`Q`](usage/#q) (short for Query) offers several ways to search and filter data, and several input modes:

---
- **<a href="../../QuickStart/usage/#q">Q.()</a>** builds a query that can be used by `run()` or `count()`
- **<a href="../../QuickStart/usage/#qrun">Q.run()</a>** returns data for the specified search 
- **<a href="../../QuickStart/usage/#qcount">Q.count()</a>** returns summary information (counts) data that fit the specified search
- **<a href="../../QuickStart/usage/#columns">columns()</a>** returns entity field names
- **<a href="../../QuickStart/usage/#unique_terms">unique_terms()</a>** returns entity field contents

---
                                                                    
Before we do any work, we need to import these functions cdapython.
We're also telling cdapython to report it's version so we can be sure we're using the one we mean to:

In [1]:
from cdapython import Q, columns, unique_terms, query
print(Q.get_version())

2022.6.28


<div class="cdanote" style="background-color:#b3e5d5;color:black;padding:20px;">
    
CDA data comes from three sources:
<ul>
<li><b>The <a href="https://proteomic.datacommons.cancer.gov/pdc/"> Proteomic Data Commons</a> (PDC)</b></li>
<li><b>The <a href="https://gdc.cancer.gov/">Genomic Data Commons</a> (GDC)</b></li>
<li><b>The <a href="https://datacommons.cancer.gov/repository/imaging-data-commons">Imaging Data Commons</a> (IDC)</b></li>
</ul> 
    
The CDA makes this data searchable in five main endpoints:

<ul>
<li><b>subject:</b> A patient entity captures the study-independent metadata for research subjects. Human research subjects are usually not traceable to a particular person to protect the subjects privacy.</li>
<li><b>researchsubject:</b> A research subject is the entity of interest in a specific research study or project, typically a human being or an animal, but can also be a device, group of humans or animals, or a tissue sample. Human research subjects are usually not traceable to a particular person to protect the subjects privacy. This entity plays the role of the case_id in existing data. A subject who participates in 3 studies will have 3 researchsubject IDs.</li>
<li><b>specimen:</b> Any material taken as a sample from a biological entity (living or dead), or from a physical object or the environment. Specimens are usually collected as an example of their kind, often for use in some investigation.</li>
<li><b>file:</b> A unit of data about subjects, researchsubjects, specimens, or their associated information.</li>
<li><b>mutation:</b> Molecular data about specific mutations, currently limited to the TCGA-READ project from GDC.</li>
</ul>
and two endpoints that offer deeper information about data in the researchsubject endpoint:
<ul>
<li><b>diagnosis:</b> A collection of characteristics that describe an abnormal condition of the body as assessed at a point in time. May be used to capture information about neoplastic and non-neoplastic conditions.</li>
<li><b>treatment:</b> Represent medication administration or other treatment types.</li>
</ul>
Any metadata field can be searched from any endpoint, the only difference between search types is what type of data is returned by default. This means that you can think of the CDA as a really, really enormous spreadsheet full of data. To search this enormous spreadsheet, you'd want select columns, and then filter rows.
</div>


If you are looking to build a cohort of distinct individuals who meet some criteria, search by `subject`. If you want to build a cohort, but are particularly interested in studies rather than the participates per se, search by `researchsubject`. If you are looking for biosamples that can be ordered or a specific format of information (for e.g. histological slides) start with `specimen`. If you are primarily looking for files you can reuse for your own analysis, start with `file`.

In CDA search, these concepts can also be chained together, so you can look specifically for specimen subjects, or researchsubject diagnoses. In the four 'main' tables, all of the rows will have one or more files associated with them that can be directly found by chaining, as in specimen files. Diagnosis and treatment do not have files directly associated with them and so can only be used to find files in conjunction with the other searches.

In all cases, any search can use any metadata field, the only difference between search types is what type of data you return by default. 



## Basic search with endpoints

Let's try a broad search of the CDA to see what information exists about cancers that were first diagnosed in the brain. To run this simple search, we would first construct a query in `Q` and save it to a variable `myquery`:

In [2]:
myquery = Q('ResearchSubject.primary_diagnosis_site = "brain"')


<div class="cdawarn" style="background-color:#f9cfbf;color:black;padding:20px;">
    
<h3>Where did those terms come from?</h3>
    
If you aren't sure how we knew what terms to put in our search, please refer back to the <a href="../SearchTerms">What search terms are available?</a> notebook. 
</div>

### subject
Now we can use that query to search any of information types. Let's start by looking at what Subjects meet our criteria. To do that, we will send our query to the subject endpoint, then ask for it to run:

In [3]:
subjectresults = myquery.subject.run()

Getting results from database

Total execution time: 3566 ms


We saved the output in a variable `subjectresults`, so we don't get much visible output. To see what our results are, we need to look into the variable. The simplest way is to call `subjectresults` directly:

In [4]:
subjectresults


            QueryID: 77faf08e-aeb1-4c83-aa00-0288493026e8
            
            Offset: 0
            Count: 100
            Total Row Count: 2384
            More pages: True
            

This tells us four parameters that describe our results:

---

- **Offset:** This is how many rows of information we've told the query to skip in the data, here we didn't tell it to skip anything, so the offset is zero
- **Count:** This is how many rows the current page of our results table has. To keep searches fast, we default to pages with 100 rows.
- **Total Row Count:** This is how many rows are in the full results table
- **More pages:** This is always a True or False. False means that our current page has all the available results. True means that we will see only the first 100 results in this table, and will need to page through for more.

---
    
We have 2384 subject matches, let's look at the actual table. The easiest way to do this is by using the python function `.to_dataframe()` on our `subjectresults` variable:

In [5]:
subjectresults.to_dataframe()

,id,identifier,species,sex,race,ethnicity,days_to_birth,subject_associated_project,vital_status,days_to_death,cause_of_death
0,C3L-03588,"[{'system': 'IDC', 'value': 'C3L-03588'}]",Homo sapiens,None,None,None,NaN,[cptac_gbm],None,NaN,None
1,C3N-03810,"[{'system': 'IDC', 'value': 'C3N-03810'}]",Homo sapiens,None,None,None,NaN,[cptac_gbm],None,NaN,None
2,C466293,"[{'system': 'PDC', 'value': 'C466293'}]",homo sapiens,male,white,not hispanic or latino,NaN,[Proteogenomic Analysis of Pediatric Brain Can...,Alive,NaN,Not Reported
3,GENIE-JHU-03295,"[{'system': 'GDC', 'value': 'GENIE-JHU-03295'}]",homo sapiens,female,black or african american,Unknown,NaN,[GENIE-JHU],Not Reported,NaN,None
4,GENIE-MDA-3866,"[{'system': 'GDC', 'value': 'GENIE-MDA-3866'}]",homo sapiens,male,white,not reported,-12053.0,[GENIE-MDA],Not Reported,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...
95,TCGA-06-0149,"[{'system': 'GDC', 'value': 'TCGA-06-0149'}, {...",homo sapiens,female,white,not hispanic or latino,-27315.0,"[tcga_gbm, TCGA-GBM]",Dead,262.0,None
96,TCGA-06-2557,"[{'system': 'GDC', 'value': 'TCGA-06-2557'}, {...",homo sapiens,male,black or african american,not hispanic or latino,-27980.0,"[tcga_gbm, TCGA-GBM]",Dead,33.0,None
97,TCGA-08-0347,"[{'system': 'GDC', 'value': 'TCGA-08-0347'}, {...",homo sapiens,male,not reported,not reported,-18384.0,"[tcga_gbm, TCGA-GBM]",Dead,782.0,None
98,TCGA-08-0354,"[{'system': 'GDC', 'value': 'TCGA-08-0354'}, {...",homo sapiens,female,white,not hispanic or latino,-19199.0,"[tcga_gbm, TCGA-GBM]",Dead,546.0,None


By default `to_dataframe()` shows us the first and last five rows for the first page of our results, so we can easily preview our data.

Since we queried the Subject endpoint, our default results tell us Subject level information, that is, information about unique individuals: their sex, race, age, species, etc. The `id` column tells us the unique identifier for each individual. The identifier column has nested information about what study or studies a Subject participated in, and will list all of their researchsubject identifiers. 

The `to_dataframe()` function converts the results to a pandas dataframe. So if we save the dataframe to a variable, we can use any pandas functions to explore it. For example, lets see whether any of the Subjects in our first 100 results are black or african american. First we'll save the results to a dataframe, then subset that dataframe to only show rows where the word "black" appears in the "race" column. "NAs" which are shown as "None" in these tables, so for our filter to work, we'll need to specifically tell it to ignore NAs:


In [6]:
subjectdata = subjectresults.to_dataframe()
subjectdata[subjectdata['race'].str.contains("black", na=False)]

,id,identifier,species,sex,race,ethnicity,days_to_birth,subject_associated_project,vital_status,days_to_death,cause_of_death
3,GENIE-JHU-03295,"[{'system': 'GDC', 'value': 'GENIE-JHU-03295'}]",homo sapiens,female,black or african american,Unknown,NaN,[GENIE-JHU],Not Reported,NaN,None
40,TCGA-DU-A5TW,"[{'system': 'GDC', 'value': 'TCGA-DU-A5TW'}, {...",homo sapiens,female,black or african american,not hispanic or latino,-12107.0,"[TCGA-LGG, tcga_lgg]",Alive,NaN,None
72,TCGA-14-3477,"[{'system': 'GDC', 'value': 'TCGA-14-3477'}, {...",homo sapiens,female,black or african american,not hispanic or latino,-14082.0,"[tcga_gbm, TCGA-GBM]",Alive,NaN,None
96,TCGA-06-2557,"[{'system': 'GDC', 'value': 'TCGA-06-2557'}, {...",homo sapiens,male,black or african american,not hispanic or latino,-27980.0,"[tcga_gbm, TCGA-GBM]",Dead,33.0,None


There are subjects in our first hundred results that meet the criteria. If we just want to be sure that the data contains some value, this might be good enough. But often we want to search the entire set of results and not just the first page. 

We'll cover how to work with large results dataframes in the <a href="../Pagination">Pagination</a> notebook. Or, learn how to get summary information from search results in the <a href="../DataSummaries">Data Summaries</a> notebook.

---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Subject Field Definitions</h3>

<i>A patient entity captures the study-independent metadata for research subjects. Human research subjects are usually not traceable to a particular person to protect the subjects privacy.</i>

    
<ul>
<li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.",STRING</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
<li><b>species:</b> The taxonomic group (e.g. species) of the subject. For MVP, since taxonomy vocabulary is consistent between GDC and PDC, using text. Ultimately, this will be a term returned by the vocabulary service.</li>
<li><b>sex:</b> The biologic character or quality that distinguishes male and female from one another as expressed by analysis of the person's gonadal, morphologic (internal and external), chromosomal, and hormonal characteristics.</li>
<li><b>race:</b> An arbitrary classification of a taxonomic group that is a division of a species. It usually arises as a consequence of geographical isolation within a species and is characterized by shared heredity, physical attributes and behavior, and in the case of humans, by common history, nationality, or geographic distribution. The provided values are based on the categories defined by the U.S. Office of Management and Business and used by the U.S. Census Bureau.</li>
<li><b>ethnicity:</b> An individual's self-described social and cultural grouping, specifically whether an individual describes themselves as Hispanic or Latino. The provided values are based on the categories defined by the U.S. Office of Management and Business and used by the U.S. Census Bureau.</li>
<li><b>days_to_birth:</b> Number of days between the date used for index and the date from a person's date of birth represented as a calculated negative number of days.</li>
<li><b>subject_associated_project:</b> The list of Projects associated with the Subject.</li>
<li><b>vital_status:</b> Coded value indicating the state or condition of being living or deceased; also includes the case where the vital status is unknown.</li>
<li><b>days_to_death:</b> Number of days between the date used for index and the date from a person's date of death represented as a calculated number of days.</li>
<li><b>cause_of_death:</b> Coded value indicating the circumstance or condition that results in the death of the subject.</li>
</ul>  

</div>
    
---

### researchsubject

If we're interested in what researchsubjects meet our criteria, we can also run our query against the researchsubject endpoint:

In [7]:
researchsubjectresults = myquery.researchsubject.run()
researchsubjectresults

Getting results from database

Total execution time: 3696 ms



            QueryID: 954e86eb-240e-40eb-a3f3-d33cb4a22c83
            
            Offset: 0
            Count: 100
            Total Row Count: 3716
            More pages: True
            

Now we see that our 2384 subjects have 3716 researchsubjects between them, that means that some, but not all, of our subjects were participants in more than one study. Let's peek at the data:

In [8]:
researchsubjectresults.to_dataframe()

,id,identifier,member_of_research_project,primary_diagnosis_condition,primary_diagnosis_site,subject_id
0,54ada9c7-e8f7-4ecc-b829-903be7e3ad82,"[{'system': 'GDC', 'value': '54ada9c7-e8f7-4ec...",GENIE-NKI,"Neoplasms, NOS",Brain,GENIE-NKI-8IN3
1,8232489b-f2dc-44a1-b6dc-38b94bda584b,"[{'system': 'GDC', 'value': '8232489b-f2dc-44a...",GENIE-JHU,"Neoplasms, NOS",Brain,GENIE-JHU-01660
2,b936c96e-70ab-46eb-a87f-73290c43f288,"[{'system': 'GDC', 'value': 'b936c96e-70ab-46e...",GENIE-NKI,"Neoplasms, NOS",Brain,GENIE-NKI-U4QV
3,be68cfb0-c47f-49a1-9e39-b0b3e57372fc,"[{'system': 'GDC', 'value': 'be68cfb0-c47f-49a...",GENIE-JHU,"Neoplasms, NOS",Brain,GENIE-JHU-02615
4,99b34e59-a5d0-4a8b-b777-3a592c5bea1b,"[{'system': 'GDC', 'value': '99b34e59-a5d0-4a8...",GENIE-JHU,"Neoplasms, NOS",Brain,GENIE-JHU-02402
...,...,...,...,...,...,...
95,6acf365b-19ce-4c97-92dd-865540a47e67,"[{'system': 'GDC', 'value': '6acf365b-19ce-4c9...",CPTAC-3,Gliomas,Brain,C3L-02642
96,94f90e9a-ecfe-4682-8d19-2ac88b31a5b9,"[{'system': 'GDC', 'value': '94f90e9a-ecfe-468...",CPTAC-3,Gliomas,Brain,C3N-01334
97,e1bd6dfa-0567-46e7-991c-63eefaf5bd7a,"[{'system': 'GDC', 'value': 'e1bd6dfa-0567-46e...",CPTAC-3,Gliomas,Brain,C3L-00674
98,142a1357-f1e8-40b9-82cf-f29577058598,"[{'system': 'GDC', 'value': '142a1357-f1e8-40b...",CPTAC-3,Gliomas,Brain,C3N-01852


Each row from the researchsubject endpoint results tells us about a subject in a given study. Using this endpoint we can find out information like what studies fit our search criteria, and also get data that we can filter to have only subjects from multiple studies, or only subjects from single studies.

Any given subject will have one row per study they participated in. The subject_id in the last column of this view is the same as the `id` in the first column of the Subjects endpoint results. You can use this to combine information across endpoints, which is covered near the end of the <a href="../BuildingACohort/#merging-results-across-endpoints">Cohort Building workflow</a> notebook.

---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>ResearchSubject Field Definitions</h3>

<i>A research subject is the entity of interest in a specific research study or project, typically a human being or an animal, but can also be a device, group of humans or animals, or a tissue sample. Human research subjects are usually not traceable to a particular person to protect the subjects privacy. This entity plays the role of the case_id in existing data. An individual who participates in 3 studies will have 3 researchsubject IDs</i>
    
<ul>
<li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system. For CDA, this is case_id.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
<li><b>member_of_research_project:</b> A reference to the Study(s) of which this ResearchSubject is a member.</li>
<li><b>primary_diagnosis_condition:</b> The text term used to describe the type of malignant disease, as categorized by the World Health Organization's (WHO) <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a> (ICD-O). This attribute represents the disease that qualified the subject for inclusion on the ResearchProject.</li>
<li><b>primary_diagnosis_site:</b> The text term used to describe the primary site of disease, as categorized by the World Health Organization's (WHO) <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a> (ICD-O). This categorization groups cases into general categories. This attribute represents the primary site of disease that qualified the subject for inclusion on the ResearchProject.</li>
<li><b>subject_id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system. Can be joined to the `id` field from subject results</li>
</ul>  

</div>
    
---

### diagnosis

The diagnosis endpoint is an extension of the researchsubject endpoint, and returns information about researchsubjects that have a diagnosis that meets our search criteria:

In [9]:
diagnosisresults = myquery.diagnosis.run()
diagnosisresults.to_dataframe()

Getting results from database

Total execution time: 3659 ms


,id,identifier,primary_diagnosis,age_at_diagnosis,morphology,stage,grade,method_of_diagnosis,subject_id,researchsubject_id
0,1e36c16c-2268-4c15-98a2-502e61ad4d84,"[{'system': 'GDC', 'value': '1e36c16c-2268-4c1...","Neoplasm, malignant",NaN,8000/3,None,Not Reported,None,GENIE-GRCC-87432b67,1a25a33b-7d0d-4fa2-bff2-8c688bea40bb
1,39b283b2-5abd-554d-8660-64d79e55455d,"[{'system': 'GDC', 'value': '39b283b2-5abd-554...",Glioblastoma,21860.0,9440/3,None,not reported,None,TCGA-32-2495,e7d728e5-1a9f-4f27-9604-272e1235e08a
2,3b3e9cd7-2848-11ec-b712-0a4e2186f121,"[{'system': 'PDC', 'value': '3b3e9cd7-2848-11e...",Glioblastoma,16667.0,9440/3,Not Reported,Not Reported,,C3L-00674,104c0976-2139-11ea-aee1-0e1aae319e49
3,3dd9d358-2848-11ec-b712-0a4e2186f121,"[{'system': 'PDC', 'value': '3dd9d358-2848-11e...",Glioblastoma,18038.0,9440/3,Not Reported,Not Reported,,C3N-00661,104c47ef-2139-11ea-aee1-0e1aae319e49
4,3e472906-2848-11ec-b712-0a4e2186f121,"[{'system': 'PDC', 'value': '3e472906-2848-11e...",Glioblastoma,16195.0,9440/3,Not Reported,Not Reported,,C3N-01798,104c5b6d-2139-11ea-aee1-0e1aae319e49
...,...,...,...,...,...,...,...,...,...,...
95,3ca82933-2848-11ec-b712-0a4e2186f121,"[{'system': 'PDC', 'value': '3ca82933-2848-11e...",Glioblastoma,22025.0,9440/3,Not Reported,Not Reported,,C3L-02705,104c291d-2139-11ea-aee1-0e1aae319e49
96,3ee64181-2848-11ec-b712-0a4e2186f121,"[{'system': 'PDC', 'value': '3ee64181-2848-11e...",Glioblastoma,17506.0,9440/3,Not Reported,Not Reported,,C3N-02782,104c74a2-2139-11ea-aee1-0e1aae319e49
97,4605156b-7d36-470b-a61c-d3bacbbccd63,"[{'system': 'GDC', 'value': '4605156b-7d36-470...",Glioblastoma,24055.0,9440/3,None,Not Reported,None,C3N-00665,3d9ab180-73ab-4938-890f-cf345bfd4cf9
98,4acbc5af-0a90-5b09-bd55-15dd2431c91c,"[{'system': 'GDC', 'value': '4acbc5af-0a90-5b0...","Oligodendroglioma, NOS",14548.0,9450/3,None,not reported,None,TCGA-HW-8322,6105c037-90f1-42d9-ac38-6e2252999a64



---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Diagnosis Field Definitions</h3>

<i>A collection of characteristics that describe an abnormal condition of the body as assessed at a point in time. May be used to capture information about neoplastic and non-neoplastic conditions.</i>

    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the repository, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>primary_diagnosis:</b> The diagnosis instance that qualified a subject for inclusion on a ResearchProject.</li>
  <li><b>age_at_diagnosis:</b> The age in days of the individual at the time of diagnosis.</li>
  <li><b>morphology:</b> Code that represents the histology of the disease using the third edition of the <a href="https://www.who.int/standards/classifications/other-classifications/international-classification-of-diseases-for-oncology">International Classification of Diseases for Oncology</a>, published in 2000, used principally in tumor and cancer registries for coding the site (topography) and the histology (morphology) of neoplasms.</li>
  <li><b>stage:</b> The extent of a cancer in the body. Staging is usually based on the size of the tumor, whether lymph nodes contain cancer, and whether the cancer has spread from the original site to other parts of the body. Different diseases may use different staging criteria, please refer to the originating data source to see what staging system is reported</li>
  <li><b>grade:</b> The degree of abnormality of cancer cells, a measure of differentiation, the extent to which cancer cells are similar in appearance and function to healthy cells of the same tissue type. The degree of differentiation often relates to the clinical behavior of the particular tumor. Based on the microscopic findings, tumor grade is commonly described by one of four degrees of severity. Histopathologic grade of a tumor may be used to plan treatment and estimate the future course, outcome, and overall prognosis of disease. Certain types of cancers, such as soft tissue sarcoma, primary brain tumors, lymphomas, and breast have special grading systems.</li>
  <li><b>method_of_diagnosis:</b> The method used to confirm the patients malignant diagnosis.</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the subject. Can be joined to the `id` field from researchsubject results</li>
</ul>  

</div>
    
---




### treatment

The treatment endpoint is an extension of diagnosis and returns information about treatments undertaken on research subjects that have a given diagnosis that meets our search criteria:

In [10]:
treatmentresults = myquery.treatment.run()
treatmentresults.to_dataframe()

Getting results from database

Total execution time: 3533 ms


,id,identifier,treatment_type,treatment_outcome,days_to_treatment_start,days_to_treatment_end,therapeutic_agent,treatment_anatomic_site,treatment_effect,treatment_end_reason,number_of_cycles,subject_id,researchsubject_id,researchsubject_diagnosis_id
0,035be2c6-6d94-5fd3-bb3c-070440d2d5b9,"[{'system': 'GDC', 'value': '035be2c6-6d94-5fd...","Pharmaceutical Therapy, NOS",None,NaN,None,None,None,None,None,None,TCGA-DU-8161,b9a3b647-61b3-4d98-90ca-e307ab50984a,5662ed5f-1e32-5699-a005-0edf7429bd3a
1,04215940-62d7-5fba-bed4-32728b696c40,"[{'system': 'GDC', 'value': '04215940-62d7-5fb...","Radiation Therapy, NOS",None,NaN,None,None,None,None,None,None,TCGA-19-0960,37a47ea2-1daa-44a4-ade5-84d383c2b13c,12bc37c7-6674-5154-8dae-f0a4e5e0096e
2,0dc4e132-d0ca-55d0-b6f6-360af92dd2a8,"[{'system': 'GDC', 'value': '0dc4e132-d0ca-55d...","Pharmaceutical Therapy, NOS",None,NaN,None,None,None,None,None,None,TCGA-FG-8182,6a0bcf0c-fa4c-4119-99d2-f722b781d20f,d5301d3a-b057-5cc2-a370-5aadad7a9b55
3,17b6d316-b324-5539-8067-c2419cd88ac8,"[{'system': 'GDC', 'value': '17b6d316-b324-553...","Pharmaceutical Therapy, NOS",None,NaN,None,None,None,None,None,None,TCGA-DU-5855,e326c1dd-2ef8-431b-a657-440aa3f35ff3,2787e928-f6e7-5053-a07a-bb1a696390fb
4,1cc2099c-0bec-5ba9-9715-438a6ad29950,"[{'system': 'GDC', 'value': '1cc2099c-0bec-5ba...","Radiation Therapy, NOS",None,NaN,None,None,None,None,None,None,TCGA-S9-A6U9,38eca720-ad68-43a7-8a1c-f79f838d5d03,f3cd7529-db2f-56f3-bb06-0cdaabf1a775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,b806fc69-ec07-5b26-8b96-dd0573dab74f,"[{'system': 'GDC', 'value': 'b806fc69-ec07-5b2...","Pharmaceutical Therapy, NOS",None,NaN,None,None,None,None,None,None,TCGA-S9-A7J3,032096a6-1923-44b9-a1cb-181bed041c8e,fc35b9d4-a743-5bca-ab74-0aa81f5b6369
96,ba38b0d3-9eab-59d8-a94c-257a5ff8d345,"[{'system': 'GDC', 'value': 'ba38b0d3-9eab-59d...","Pharmaceutical Therapy, NOS",None,NaN,None,None,None,None,None,None,TCGA-19-0963,163e93a7-4d8d-4f28-a9f6-4d0ed16630cd,45ae8847-a98e-5bea-a7b8-31f78c5e09a8
97,ce366c44-ae99-590f-af64-c98a5b17d13b,"[{'system': 'GDC', 'value': 'ce366c44-ae99-590...","Pharmaceutical Therapy, NOS",None,NaN,None,None,None,None,None,None,TCGA-08-0389,ea980264-d64d-410a-86dd-c129d7d8d1ee,96944e9a-22f6-5ac4-9ae7-1cd512297d3e
98,ce369bc9-a7ce-5b9e-a1cd-cba77c86dbc6,"[{'system': 'GDC', 'value': 'ce369bc9-a7ce-5b9...","Radiation Therapy, NOS",None,NaN,None,None,None,None,None,None,TCGA-S9-A7R4,58650d08-7685-41de-9ae0-21665f8f449f,bcac39f9-4ba0-599e-85b8-a529e361d038



---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Treatment Field Definitions</h3>

<i><i> Medication administration or other treatment types. A single research subject may have multiple treatments for a single diagnosis, and/or different diagnoses, and different treatments, across different studies</i></i>

    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the repository, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>treatment_type:</b> The treatment type including medication/therapeutics or other procedures.</li>
  <li><b>treatment_outcome:</b> The final outcome of the treatment.</li>
  <li><b>days_to_treatment_start:</b> The timepoint at which the treatment started.</li>
  <li><b>days_to_treatment_end:</b>The timepoint at which the treatment ended. </li>
  <li><b>therapeutic_agent:</b> One or more therapeutic agents as part of this treatment.</li>
  <li><b>treatment_anatomic_site:</b> The anatomical site that the treatment targets.</li>
  <li><b>treatment_effect:</b>The effect of a treatment on the diagnosis or tumor. </li>
  <li><b>treatment_end_reason:</b>The reason the treatment ended. </li>
  <li><b>number_of_cycles:</b>The number of treatment cycles the subject received. </li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the researchsubject. Can be joined to the `id` field from researchsubject results</li>
  <li><b>researchsubject_diagnosis_id:</b> An identifier for the diagnosis. Can be joined to the `id` field from diagnosis results</li>
</ul>  

</div>
    
---




### specimen

We can use this same query to see what specimens are available for brain tissue at the CDA:

In [11]:
specimenresults =  myquery.specimen.run()
print(specimenresults)

Getting results from database

Total execution time: 4069 ms

            QueryID: fdf0c44f-5d00-4917-83f4-282484d4c971
            
            Offset: 0
            Count: 100
            Total Row Count: 39201
            More pages: True
            


Nearly 40,000 specimens meet our search criteria! We would typically expect this number to be much larger than our number of subjects or researchsubjects. First because studies will often take more than one sample per subject, and second because any given specimen might be aliquoted out to be used in multiple tests. Since we didn't specify any further filters, our results will return all of these as separate speciments. Let's look at a few:

In [12]:
specimenresults.to_dataframe()

,id,identifier,associated_project,days_to_collection,primary_disease_type,anatomical_site,source_material_type,specimen_type,derived_from_specimen,subject_id,researchsubject_id
0,0072c6f0-c876-4296-aabe-376ef49efab0,"[{'system': 'GDC', 'value': '0072c6f0-c876-429...",TCGA-GBM,NaN,Gliomas,None,Primary Tumor,analyte,f2b1a88f-baf0-4ddc-a527-b0fdc08c5145,TCGA-02-0046,5b8f05fa-0145-4142-8d41-5980e4d36d81
1,01e4984a-e570-4623-b020-2ab576af9727,"[{'system': 'GDC', 'value': '01e4984a-e570-462...",TCGA-LGG,0.0,Gliomas,None,Primary Tumor,slide,4f41877a-1b43-52c0-81cd-c7de3fc7e1da,TCGA-S9-A6WI,0b283f95-dbd4-4c36-9201-2198a90ee0d6
2,025bebe3-cf8e-4c92-92b4-39bd4003f6ab,"[{'system': 'GDC', 'value': '025bebe3-cf8e-4c9...",TCGA-LGG,0.0,Gliomas,None,Primary Tumor,slide,7dc77a07-bbc3-5056-9e39-016379e326a2,TCGA-R8-A6MK,65836b59-b469-4b89-adf5-e65735e494f5
3,029fdf74-3d3a-48a6-b2b0-76d10fe7fd59,"[{'system': 'GDC', 'value': '029fdf74-3d3a-48a...",TCGA-LGG,NaN,Gliomas,None,Blood Derived Normal,aliquot,94ee050b-69be-422a-bc95-cec676e445f8,TCGA-HT-8105,d766c3af-f0b3-4791-a754-d6546d370f8f
4,032dd99e-66bf-4b29-b9e9-952289f28d6a,"[{'system': 'GDC', 'value': '032dd99e-66bf-4b2...",HCMI-CMDC,NaN,Gliomas,None,Primary Tumor,portion,be4394ef-8a86-4eec-8b76-760773a23253,HCM-BROD-0613-C71,b3cc0754-6a21-4683-a491-b2fb8b6175a4
...,...,...,...,...,...,...,...,...,...,...,...
95,3d4a149d-0b92-402c-af45-94675bee4710,"[{'system': 'GDC', 'value': '3d4a149d-0b92-402...",TCGA-GBM,NaN,Gliomas,None,Primary Tumor,aliquot,9d7d0534-0230-4af2-8df4-2adda1966077,TCGA-14-1451,06c427a1-8c2d-4003-b744-139114a04440
96,3df80cf4-56dd-48e0-9112-b0d730bdb2e9,"[{'system': 'GDC', 'value': '3df80cf4-56dd-48e...",TCGA-GBM,NaN,Gliomas,None,Primary Tumor,aliquot,47959142-1c3d-4764-8dbc-bcd91612f47d,TCGA-27-2528,be3a7ef3-34ed-40e1-9d9c-187940596b26
97,3ec82364-e5f1-41d3-a4ec-9e81f7616165,"[{'system': 'GDC', 'value': '3ec82364-e5f1-41d...",TCGA-GBM,NaN,Gliomas,None,Solid Tissue Normal,analyte,863a25cc-8d7e-440e-b98d-72941e53a313,TCGA-08-0359,0aa5b68f-2b99-44d2-b17f-f4eb17fe891e
98,3f20d55d-f1ad-4ede-9905-b456d7db4898,"[{'system': 'GDC', 'value': '3f20d55d-f1ad-4ed...",TCGA-LGG,1243.0,Gliomas,None,Primary Tumor,slide,1e57b685-0cd6-421a-8e8e-c2f3820642f4,TCGA-TM-A84G,63f4aa26-dec0-4309-8b75-6446a7241527



---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>Specimen Field Definitions</h3>

<i><i>Any material taken as a sample from a biological entity (living or dead), or from a physical object or the environment. Specimens are usually collected as an example of their kind, often for use in some investigation.</i>
 A given specimen will have only a single subject ID and a single research subject ID</i>
    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the repository, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>associated_project:</b> The Project associated with the specimen.</li>
  <li><b>days_to_collection:</b> The number of days from the index date to either the date a sample was collected for a specific study or project, or the date a patient underwent a procedure (e.g. surgical resection) yielding a sample that was eventually used for research.</li>
  <li><b>primary_disease_type:</b> The text term used to describe the type of malignant disease, as categorized by the World Health Organization's (WHO) International Classification of Diseases for Oncology (ICD-O). This attribute represents the disease that qualified the subject for inclusion on the ResearchProject.</li>
  <li><b>anatomical_site:</b> Per GDC Dictionary, the text term that represents the name of the primary disease site of the submitted tumor sample; recommend dropping tumor; biospecimen_anatomic_site.</li>
  <li><b>source_material_type:</b> The general kind of material from which the specimen was derived, indicating the physical nature of the source material.</li>
  <li><b>specimen_type:</b> The high-level type of the specimen, based on its how it has been derived from the original extracted sample. One of: analyte, aliquot, portion, sample, or slide.</li>
  <li><b>derived_from_specimen:</b> A source/parent specimen from which this one was directly derived.</li>
  <li><b>subject_id:</b> An identifier for the subject. Can be joined to the `id` field from subject results</li>
  <li><b>researchsubject_id:</b> An identifier for the subject. Can be joined to the `id` field from researchsubject results</li>
</ul>  

</div>
    
---


### file

The file endpoint returns information about files that meet our search criteria, regardless of whether they are attached to subjects, research-subjects or specimens: 

In [13]:
myquery.file.run()

Getting results from database

Total execution time: 3858 ms



            QueryID: 10be2d40-2d6e-4b23-90ee-a67353c088ee
            
            Offset: 0
            Count: 100
            Total Row Count: 4138879
            More pages: True
            

In [14]:
fileresults = myquery.file.run()
fileresults.to_dataframe()

Getting results from database

Total execution time: 3764 ms


,id,identifier,label,data_category,data_type,file_format,associated_project,drs_uri,byte_size,checksum,data_modality,imaging_modality,dbgap_accession_number,imaging_series,researchsubject_specimen_id,researchsubject_id,subject_id
0,e5b8f4c3-efa1-412c-8769-6bd629ceff25,"[{'system': 'PDC', 'value': 'e5b8f4c3-efa1-412...",09CPTAC_GBM_W_PNNL_20190501_B3S1_f24.mzid.gz,Peptide Spectral Matches,Open Standard,mzIdentML,CPTAC3-Discovery,drs://dg.4DFC:e5b8f4c3-efa1-412c-8769-6bd629ce...,8445510,b707f6f6abb880553146af8214911d06,Proteomic,None,None,None,,,C3N-01366
1,416a6c7e-15e5-11ea-9bfa-0a42f3c845fe,"[{'system': 'PDC', 'value': '416a6c7e-15e5-11e...",06CPTAC_GBM_P_PNNL_20190306_B2S2_f12.raw,Raw Mass Spectra,Proprietary,vendor-specific,CPTAC3-Discovery,drs://dg.4DFC:416a6c7e-15e5-11ea-9bfa-0a42f3c8...,615428786,e2620190ade98fff9bffd0149feefd8d,Proteomic,None,None,None,,,C3N-02185
2,32a4267d-0cc1-4022-ae45-5efb4a29eef6,"[{'system': 'PDC', 'value': '32a4267d-0cc1-402...",08CPTAC_GBM_W_PNNL_20190306_B2S4_f18.mzML.gz,Processed Mass Spectra,Open Standard,mzML,CPTAC3-Discovery,drs://dg.4DFC:32a4267d-0cc1-4022-ae45-5efb4a29...,200563993,170fca4126b7a4977c5d2248f0b9b537,Proteomic,None,None,None,,,C3N-01856
3,6301f865-15e6-11ea-9bfa-0a42f3c845fe,"[{'system': 'PDC', 'value': '6301f865-15e6-11e...",11CPTAC_GBM_W_PNNL_20190501_B3S3_f09.raw,Raw Mass Spectra,Proprietary,vendor-specific,CPTAC3-Discovery,drs://dg.4DFC:6301f865-15e6-11ea-9bfa-0a42f3c8...,828570494,d3e410d4c2188a45db3d0890f0329d00,Proteomic,None,None,None,,,C3N-02786
4,77cfc916-814e-48c9-8a4c-689486c7d0a4,"[{'system': 'PDC', 'value': '77cfc916-814e-48c...",NCI-11Plex-23-F8-m17322.mzML.gz,Processed Mass Spectra,Open Standard,mzML,Proteogenomic Analysis of Pediatric Brain Canc...,drs://dg.4DFC:77cfc916-814e-48c9-8a4c-689486c7...,190616400,5b2f6ec7c4688f49e45ec896bbfe7eeb,Proteomic,None,None,None,,,C58917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,b1146352-9ebd-4236-8f61-baece7f6af59,"[{'system': 'PDC', 'value': 'b1146352-9ebd-423...",01CPTAC_GBM_P_PNNL_20190123_B1S1_f03.mzid.gz,Peptide Spectral Matches,Open Standard,mzIdentML,CPTAC3-Discovery,drs://dg.4DFC:b1146352-9ebd-4236-8f61-baece7f6...,5394853,7f78d332dfa2daaef3452c62c934a16b,Proteomic,None,None,None,,,C3N-02256
96,a9fffc9c-97cc-458d-bff4-c8fc814f3c74,"[{'system': 'PDC', 'value': 'a9fffc9c-97cc-458...",NCI-11pex-8H-Phos-JM05257.mzid.gz,Peptide Spectral Matches,Open Standard,mzIdentML,Proteogenomic Analysis of Pediatric Brain Canc...,drs://dg.4DFC:a9fffc9c-97cc-458d-bff4-c8fc814f...,3171775,de574a09f9e8b76a73dd6f1fb4edc2c9,Proteomic,None,None,None,,,C20541
97,b0a661d0-312c-4ee0-bc49-66ef7eb95294,"[{'system': 'PDC', 'value': 'b0a661d0-312c-4ee...",10CPTAC_GBM_P_PNNL_20190501_B3S2_f01.psm,Peptide Spectral Matches,Text,tsv,CPTAC3-Discovery,drs://dg.4DFC:b0a661d0-312c-4ee0-bc49-66ef7eb9...,2179691,af132b8401965079a0d3865acb150f99,Proteomic,None,None,None,,,C3N-03184
98,621b178a-496a-4949-a609-a598bc6abc37,"[{'system': 'PDC', 'value': '621b178a-496a-494...",NCI-11Plex-3-F4-f10331.mzML.gz,Processed Mass Spectra,Open Standard,mzML,Proteogenomic Analysis of Pediatric Brain Canc...,drs://dg.4DFC:621b178a-496a-4949-a609-a598bc6a...,159363389,55884838627eb0f3c79ae9d01f2faf40,Proteomic,None,None,None,,,C30258


As you might expect, searching file gives us a huge number of results. This is great if you are surveying what kind of data is available, but is less useful for getting a coherent cohort. 

A better way to get files for a specific cohort is to chain your queries together, which we cover in the next tutorial <a href="../Chaining">Chaining Endpoints</a>: Combine information from multiple endpoints, and build And/Or/Like and other advanced query strings.

Another useful way to look at high level information is to use our counts feature which returns summary information rather than the full search results. Check out the <a href="../DataSummaries">Data Summaries tutorial</a> to try it.



---

<div class="cdadefine" style="background-color:#add9e5;color:black;padding:20px;">

<h3>File Field Definitions</h3>

<i>A file is an information-bearing electronic object that contains a physical embodiment of some information using a particular character encoding.</i>

    
<ul>
  <li><b>id:</b> The 'logical' identifier of the entity in the system of record, e.g. a UUID. This 'id' is unique within a given system. The identified entity may have a different 'id' in a different system.</li>
<li><b>identifier:</b> A 'business' identifier for the entity, typically as provided by an external system or authority, that persists across implementing systems (i.e. a 'logical' identifier). Uses a specialized, complex 'Identifier' data type to capture information about the source of the business identifier - or a URI expressed as a string to an existing entity.</li>
<li><b>identifier.system:</b> The system or namespace that defines the identifier.</li>
<li><b>identifier.value:</b> The value of the identifier, as defined by the system.</li>
  <li><b>label:</b> Short name or abbreviation for dataset. Maps to rdfs:label.</li>
  <li><b>data_catagory:</b> Broad categorization of the contents of the data file.</li>
  <li><b>data_type:</b> Specific content type of the data file.</li>
  <li><b>file_format:</b> Format of the data files.</li>
  <li><b>associated_project:</b> A reference to the Project(s) of which this ResearchSubject is a member. The associated_project may be embedded using the ref definition or may be a reference to the id for the Project - or a URI expressed as a string to an existing entity.</li>
  <li><b>drs_uri:</b> A string of characters used to identify a resource on the Data Repo Service(DRS). Can be used to retrieve this specific file from a server.</li>
  <li><b>byte_size:</b> Size of the file in bytes. Maps to dcat:byteSize.</li>
  <li><b>checksum:</b> The md5 value for the file. A digit representing the sum of the correct digits in a piece of stored or transmitted digital data, against which later comparisons can be made to detect errors in the data.</li>
  <li><b>data_modality:</b> Data modality describes the biological nature of the information gathered as the result of an Activity, independent of the technology or methods used to produce the information. Always one of "Genomic", "Proteomic", or "Imaging".</li>
  <li><b>imaging_modality:</b> An imaging modality describes the imaging equipment and/or method used to acquire certain structural or functional information about the body. These include but are not limited to computed tomography (CT) and magnetic resonance imaging (MRI). Taken from the DICOM standard.</li>
  <li><b>dbgap_accession_number:</b> The dbgap accession number for the project.</li>
</ul>  

</div>
    
---